In [15]:
#pip install beautifulsoup4
#pip install requests
#pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim
#from geopy.exc import GeocoderTimedOut
import re

In [2]:
from geopy.exc import GeocoderTimedOut

In [3]:
# The URL of the website you want to scrape
url = 'https://www.trouvetamosquee.fr/rechercher-une-mosquee/?cname=*'  # Replace this with the desired website URL

# Send an HTTP request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Now, you can work with the 'soup' object to extract information from the website
    # For example, find and print the title of the webpage
    print("Title:", soup.title.get_text())
    
    # Find a specific element by its tag, class, id, etc.
    specific_element = soup.find('div', class_='specific-class')  # Replace with your desired element and its attributes
    if specific_element:
        print("Specific Element:", specific_element)

    # You can extract more information as needed by navigating the HTML structure using Beautiful Soup

else:
    print("Failed to retrieve the webpage")

Title: Rechercher une mosquée – Trouve Ta Mosquée


In [4]:
df = pd.DataFrame(columns = ['Name', 'Website', 'Adress', 'Dpt. Code', 'Dpt. Name', 'Phone', 'Latitude', 'Longitude'])

In [5]:
loc = Nominatim(user_agent="Geopy Library")

In [6]:
mosqueelist = soup.find('article', class_='mosque-list').find('ul').find_all('li')

In [119]:
#for mosquee in mosqueelist:
#    Name = mosquee.find('h2').find('a').get_text()
#    Website = mosquee.find('h2').find('a').get('href')
#    Adress = mosquee.find('div', class_='list-content clearfix').find('div', class_='content-details left').find('h3', class_='address').find('a').get_text().split(" - ")[0]
#    DptCode = mosquee.find('div', class_='list-content clearfix').find('div', class_='content-details left').find('h3', class_='address').find('a').get_text().split(" - ")[1].split(" ")[0]
#    DptName = mosquee.find('div', class_='list-content clearfix').find('div', class_='content-details left').find('h3', class_='address').find('a').get_text().split(" - ")[1].split(" ")[1]
#    if mosquee.find('div', class_='list-content clearfix').find('div', class_='content-details left').find('p', class_='phone'):
#        Phone = mosquee.find('div', class_='list-content clearfix').find('div', class_='content-details left').find('p', class_='phone').get_text()[12:]
#    else:
#        Phone = "Not found" 
#    
#    df = df.append({'Name' : Name, 'Website' : Website, 'Adress' : Adress, 'Dpt. Code' : DptCode, 'Dpt. Name' : DptName, 'Phone' : Phone}, ignore_index = True)

In [33]:
for idx, mosquee in enumerate(mosqueelist):
    Name = mosquee.find('h2').find('a').get_text()
    Website = mosquee.find('h2').find('a').get('href')
    Adress = mosquee.find('div', class_='list-content clearfix').find('div', class_='content-details left').find('h3', class_='address').find('a').get_text().split(" - ", 1)[0]
    DptCode = mosquee.find('div', class_='list-content clearfix').find('div', class_='content-details left').find('h3', class_='address').find('a').get_text().split(" - ", 1)[1].split(" ", 1)[0]
    DptName = mosquee.find('div', class_='list-content clearfix').find('div', class_='content-details left').find('h3', class_='address').find('a').get_text().split(" - ", 1)[1].split(" ", 1)[1]
    if mosquee.find('div', class_='list-content clearfix').find('div', class_='content-details left').find('p', class_='phone'):
        Phone = mosquee.find('div', class_='list-content clearfix').find('div', class_='content-details left').find('p', class_='phone').get_text()[12:]
    else:
        Phone = "Not found"
    
    try:
        if loc.geocode(Adress+', '+ DptName):
            getLoc = loc.geocode(Adress+', '+ DptName)
            Lati = getLoc.latitude
            Longi = getLoc.longitude
        elif loc.geocode(Adress+', '+ DptCode):
            getLoc = loc.geocode(Adress+', '+ DptCode)
            Lati = getLoc.latitude
            Longi = getLoc.longitude
        elif loc.geocode(Adress+', '+ DptName +', '+ DptCode):
            getLoc = loc.geocode(Adress+', '+ DptName +', '+ DptCode)
            Lati = getLoc.latitude
            Longi = getLoc.longitude
        elif loc.geocode(Adress+', '+ DptName +', '+ DptCode +', '+ "France"):
            getLoc = loc.geocode(Adress+', '+ DptName +', '+ DptCode +', '+ "France")
            Lati = getLoc.latitude
            Longi = getLoc.longitude
        else:
            Lati = 'Not found'
            Longi = 'Not found'
    except:
        Lati = 'Not found'
        Longi = 'Not found'
    
    #if idx != 615:
    #    if loc.geocode(Adress+', '+ DptName):
    #        getLoc = loc.geocode(Adress+', '+ DptName)
    #        Lati = getLoc.latitude
    #        Longi = getLoc.longitude
    #    else:
    #        Lati = 'Not found'
    #        Longi = 'Not found'
    #else:
    #    Lati = 'Not found'
    #    Longi = 'Not found'
    
    df = df.append({'Name' : Name, 'Website' : Website, 'Adress' : Adress, 'Dpt. Code' : DptCode, 'Dpt. Name' : DptName, 'Phone' : Phone, 'Latitude' : Lati, 'Longitude' : Longi}, ignore_index = True)

In [57]:
#mosqueelist.find('li').find('h2').find('a').get_text()
#mosqueelist.find('li').find('h2').find('a').get('href')
#name = mosqueelist.find('li').find('div', class_='list-content clearfix').find('div', class_='content-details left').find('h3', class_='address').find('a').get_text()
#name.split(" - ")[0]
#name.split(" - ")[1].split(" ")
#mosqueelist.find('li').find('div', class_='list-content clearfix').find('div', class_='content-details left').find('p', class_='phone').get_text()[12:]

'Mosquée Sultan Ahmet Camii'

In [37]:
df.to_excel('/Users/dantoni/Desktop/Mouad.xlsx', index=False)